In [ ]:
from deep_speaker.conv_models import DeepSpeakerModel
from glob import glob
import os
import librosa
import random

import numpy as np

from deep_speaker.audio import read_mfcc
from deep_speaker.batcher import sample_from_mfcc
from deep_speaker.constants import SAMPLE_RATE, NUM_FRAMES
from deep_speaker.conv_models import DeepSpeakerModel
from deep_speaker.test import batch_cosine_similarity

In [ ]:
model = DeepSpeakerModel()
path = '/home/repsmate/saa-sr/checkpoints-triplets/ResCNN_checkpoint_120.h5'

# Load the checkpoint. https://drive.google.com/file/d/1F9NvdrarWZNktdX9KlRYWWHDwRkip_aP.
# Also available here: https://share.weiyun.com/V2suEUVh (Chinese users).
model.m.load_weights(path, by_name=True)

In [ ]:
audio_dir = '/home/repsmate/datasets/test'
config_path = '/home/repsmate/saa-sr-inference/tests/config.yaml'
audio_files = glob(os.path.join(audio_dir, '*.wav'))

In [ ]:
mfcc_001 = sample_from_mfcc(read_mfcc(audio_files[0], SAMPLE_RATE), NUM_FRAMES)
mfcc_002 = sample_from_mfcc(read_mfcc(audio_files[1], SAMPLE_RATE), NUM_FRAMES)
predict_001 = model.m.predict(np.expand_dims(mfcc_001, axis=0))
predict_002 = model.m.predict(np.expand_dims(mfcc_002, axis=0))

In [ ]:
score_pos = batch_cosine_similarity(np.concatenate([predict_001, predict_001, predict_002, predict_002]), np.concatenate([predict_001, predict_002, predict_001, predict_002]))
print(score_pos)

In [ ]:
ref_voice_features = np.concatenate([predict_002, predict_002, predict_001, predict_002, predict_002, predict_002, predict_002, predict_002])
voice_features = np.concatenate([predict_001, predict_002, predict_002, predict_002, predict_002, predict_001])
np.mean(np.dot(voice_features, ref_voice_features.T), axis=1)

In [ ]:
score_neg = score_pos.copy()
score_neg[0] = 1
score_pos < score_neg, score_pos, score_neg

In [ ]:
print(predict_001.shape) [0.89747113]

In [ ]:
batch_cosine_similarity(predict_001, predict_002)

In [ ]:
scores = []
for audio_file in audio_files:
    mfcc = sample_from_mfcc(read_mfcc(audio_file, SAMPLE_RATE), NUM_FRAMES)
    predict = model.m.predict(np.expand_dims(mfcc, axis=0))
    score = batch_cosine_similarity(predict_001, predict)
    scores.append(score)



In [ ]:
from scipy.io import wavfile

In [ ]:
len(scores), len(audio_files), np.shape(scores)

In [ ]:
thr = 0.87
scores = np.squeeze(np.array(scores))
audio_files = np.array(audio_files)
spk1_files = audio_files[scores > thr]
spk2_files = audio_files[scores <= thr]

spk1_data, spk2_data = [], []
for spk1_file in spk1_files:
    sr, data = wavfile.read(spk1_file)
    spk1_data.append(data)
for spk2_file in spk2_files:
    sr, data = wavfile.read(spk2_file)
    spk2_data.append(data)
spk1_data = np.concatenate(spk1_data)
spk2_data = np.concatenate(spk2_data)

wavfile.write('/home/repsmate/ceva1.wav', 16000, spk1_data.astype(np.int16))
wavfile.write('/home/repsmate/ceva2.wav', 16000, spk1_data.astype(np.int16))


In [ ]:
from matplotlib.pyplot import plot

In [ ]:
plot(scores)